In [1]:
import pandas as pd
import numpy as np
import keras
import tensorflow
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Embedding,Flatten,Dense


In [2]:
data = pd.read_csv("text_emotions/train.txt", sep=';')
data.columns = ["Text", "Emotions"]
print(data.head())

                                                Text Emotions
0  i can go from feeling so hopeless to so damned...  sadness
1   im grabbing a minute to post i feel greedy wrong    anger
2  i am ever feeling nostalgic about the fireplac...     love
3                               i am feeling grouchy    anger
4  ive been feeling a little burdened lately wasn...  sadness


In [3]:
texts = data["Text"].tolist()
labels = data["Emotions"].tolist()

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

In [4]:
sequences = tokenizer.texts_to_sequences(texts)
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length)

In [5]:
# Encode the string labels to integers
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

In [6]:
# One-hot encode the labels
one_hot_labels = keras.utils.to_categorical(labels)

In [7]:
# Split the data into training and testing sets
xtrain, xtest, ytrain, ytest = train_test_split(padded_sequences,
                                                one_hot_labels,
                                                test_size=0.2)

In [8]:
# Define the model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1,
                    output_dim=128, input_length=max_length))
model.add(Flatten())
model.add(Dense(units=128, activation="relu"))
model.add(Dense(units=len(one_hot_labels[0]), activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(xtrain, ytrain, epochs=10, batch_size=32, validation_data=(xtest, ytest))

Epoch 1/10
400/400 [==============================] - 25s 58ms/step - loss: 1.3884 - accuracy: 0.4719 - val_loss: 0.9442 - val_accuracy: 0.6609
Epoch 2/10
400/400 [==============================] - 22s 56ms/step - loss: 0.4087 - accuracy: 0.8751 - val_loss: 0.5224 - val_accuracy: 0.8178
Epoch 3/10
400/400 [==============================] - 24s 59ms/step - loss: 0.0747 - accuracy: 0.9808 - val_loss: 0.5858 - val_accuracy: 0.8163
Epoch 4/10
400/400 [==============================] - 21s 53ms/step - loss: 0.0282 - accuracy: 0.9942 - val_loss: 0.5648 - val_accuracy: 0.8288
Epoch 5/10
400/400 [==============================] - 24s 59ms/step - loss: 0.0198 - accuracy: 0.9962 - val_loss: 0.6002 - val_accuracy: 0.8234
Epoch 6/10
400/400 [==============================] - 22s 55ms/step - loss: 0.0163 - accuracy: 0.9964 - val_loss: 0.6436 - val_accuracy: 0.8166
Epoch 7/10
400/400 [==============================] - 22s 54ms/step - loss: 0.0149 - accuracy: 0.9966 - val_loss: 0.6600 - val_accuracy:

In [9]:
input_text = "i feel a little mellow today"

# Preprocess the input text
input_sequence = tokenizer.texts_to_sequences([input_text])
padded_input_sequence = pad_sequences(input_sequence, maxlen=max_length)
prediction = model.predict(padded_input_sequence)
predicted_label = label_encoder.inverse_transform([np.argmax(prediction[0])])
print(predicted_label)

1/1 [==============================] - 0s 154ms/step
['joy']
